# HW 4-2 Regression
20220041

In [56]:
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print(train_data.info())

train_feature = train_data.drop(columns=['MIN', 'position'])
train_label = train_data['MIN']

print("Training features:\n", train_feature.head())
print("Training label:\n", train_label.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SEASON_ID   15000 non-null  object 
 1   TEAM_ID     15000 non-null  int64  
 2   PLAYER_AGE  15000 non-null  float64
 3   GP          15000 non-null  int64  
 4   GS          11838 non-null  float64
 5   MIN         14610 non-null  float64
 6   FGM         15000 non-null  int64  
 7   FGA         15000 non-null  int64  
 8   FG_PCT      14994 non-null  float64
 9   FG3M        12057 non-null  float64
 10  FG3A        12057 non-null  float64
 11  FG3_PCT     11955 non-null  float64
 12  FTM         15000 non-null  int64  
 13  FTA         15000 non-null  int64  
 14  FT_PCT      14952 non-null  float64
 15  OREB        12931 non-null  float64
 16  DREB        12931 non-null  float64
 17  REB         14662 non-null  float64
 18  AST         15000 non-null  int64  
 19  STL         12931 non-nul

Data preprocessing

1. Data selection

In [57]:
# Remove attributes that are not included in the test data
extra_cols = set(train_feature.columns) - set(test_data.columns)
print(extra_cols)
train_data = train_data.drop(columns=extra_cols)
print(train_data.info())

{'GS', 'GP'}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SEASON_ID   15000 non-null  object 
 1   TEAM_ID     15000 non-null  int64  
 2   PLAYER_AGE  15000 non-null  float64
 3   MIN         14610 non-null  float64
 4   FGM         15000 non-null  int64  
 5   FGA         15000 non-null  int64  
 6   FG_PCT      14994 non-null  float64
 7   FG3M        12057 non-null  float64
 8   FG3A        12057 non-null  float64
 9   FG3_PCT     11955 non-null  float64
 10  FTM         15000 non-null  int64  
 11  FTA         15000 non-null  int64  
 12  FT_PCT      14952 non-null  float64
 13  OREB        12931 non-null  float64
 14  DREB        12931 non-null  float64
 15  REB         14662 non-null  float64
 16  AST         15000 non-null  int64  
 17  STL         12931 non-null  float64
 18  BLK         12931 non-null  float64
 19  TOV         

2. Data cleaning (Handling missing values)

In [58]:
# Display rows with any missing values in the training data
rows_with_missing_values = train_data[train_data.isnull().any(axis=1)]

print("Rows with missing values:")
print(rows_with_missing_values)

Rows with missing values:
      SEASON_ID     TEAM_ID  PLAYER_AGE     MIN  FGM   FGA  FG_PCT  FG3M  \
2       1964-65  1610612765        28.0   753.0   84   221   0.380   NaN   
6       1977-78  1610612762        25.0  2119.0  304   602   0.505   NaN   
15      1978-79  1610612765        22.0  2560.0  456   946   0.482   NaN   
17      1946-47  1610610035        23.0     NaN   58   260   0.223   NaN   
25      1973-74  1610612739        22.0  1862.0  210   548   0.383   NaN   
...         ...         ...         ...     ...  ...   ...     ...   ...   
14979   1976-77  1610612755        25.0  2037.0  426   823   0.518   NaN   
14981   1963-64           0        24.0   147.0   12    30   0.400   NaN   
14986   1962-63  1610612765        27.0  2961.0  636  1450   0.439   NaN   
14992   1973-74  1610612745        30.0    38.0    7    12   0.583   NaN   
14996   1970-71  1610612747        36.0    57.0    8    19   0.421   NaN   

       FG3A  FG3_PCT  ...   OREB   DREB    REB  AST    STL   

In [59]:
# Fill the missing values
train_feature = train_data.drop(columns=['MIN', 'position'])
train_label = train_data['MIN']

categorical_cols = train_feature.select_dtypes(include=['object']).columns
numerical_cols = train_feature.select_dtypes(include=['int64', 'float64']).columns

train_feature[numerical_cols] = train_feature[numerical_cols].fillna(train_feature[numerical_cols].median())

for col in categorical_cols:
    train_feature[col] = train_feature[col].fillna(train_feature[col].mode()[0])

train_label = train_label.fillna(train_label.median())
print(train_feature.info())
print(train_label.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SEASON_ID   15000 non-null  object 
 1   TEAM_ID     15000 non-null  int64  
 2   PLAYER_AGE  15000 non-null  float64
 3   FGM         15000 non-null  int64  
 4   FGA         15000 non-null  int64  
 5   FG_PCT      15000 non-null  float64
 6   FG3M        15000 non-null  float64
 7   FG3A        15000 non-null  float64
 8   FG3_PCT     15000 non-null  float64
 9   FTM         15000 non-null  int64  
 10  FTA         15000 non-null  int64  
 11  FT_PCT      15000 non-null  float64
 12  OREB        15000 non-null  float64
 13  DREB        15000 non-null  float64
 14  REB         15000 non-null  float64
 15  AST         15000 non-null  int64  
 16  STL         15000 non-null  float64
 17  BLK         15000 non-null  float64
 18  TOV         15000 non-null  float64
 19  PF          15000 non-nul

3. Data transformation

In [60]:
train_feature['SEASON_ID'] = train_feature['SEASON_ID'].str[:4].astype(int)

print("Number of rows in train_feature:", len(train_feature))
print("Number of rows in train_label:", len(train_label))

Number of rows in train_feature: 15000
Number of rows in train_label: 15000


Training data

In [61]:
from sklearn.model_selection import train_test_split

feature_train, feature_val, label_train, label_val = train_test_split(
    train_feature, 
    train_label, 
    test_size=0.2,
    random_state=10000
)

1. Linear regression

In [62]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

linear_model = LinearRegression()
linear_model.fit(feature_train, label_train)

label_pred = linear_model.predict(feature_val)
mse = mean_squared_error(label_val, label_pred)
print("Linear Regression MSE:", mse)

Linear Regression MSE: 46286.21900153978


2. KNN

In [63]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

knn_regressor = KNeighborsRegressor(n_neighbors=5)
knn_regressor.fit(feature_train, label_train)

label_pred_knn = knn_regressor.predict(feature_val)
mse_knn = mean_squared_error(label_val, label_pred_knn)
print("KNN Regression MSE:", mse_knn)

KNN Regression MSE: 39926.71634666666


Hyperparameter tuning

In [64]:
from sklearn.model_selection import GridSearchCV

param_grid_knn = {
    'n_neighbors': [1, 3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance']
}

grid_search_knn = GridSearchCV(
    KNeighborsRegressor(), 
    param_grid_knn, 
    cv=5, 
    scoring='neg_mean_squared_error'
)
grid_search_knn.fit(feature_train, label_train)

print(grid_search_knn.best_params_)

{'n_neighbors': 9, 'weights': 'distance'}


In [65]:
# Prediction with the best knn
best_knn = grid_search_knn.best_estimator_
label_pred_knn = best_knn.predict(feature_val)

mse_knn = mean_squared_error(label_val, label_pred_knn)
print("Best KNN Regression MSE:", mse_knn)

Best KNN Regression MSE: 37905.01715800408


Prediction on the test data

In [66]:
test_data_clean = test_data.drop(columns=['ID'])
test_data_clean['SEASON_ID'] = test_data_clean['SEASON_ID'].str[:4].astype(int)

test_pred =  best_knn.predict(test_data_clean)

result = pd.DataFrame({
    'ID': test_data['ID'],
    'MIN': test_pred
})

result.to_csv('reg3.csv', index=False)
print("Predictions saved")

Predictions saved
